## - - 이전 코드 시작 - -

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 547 kB/s 
     |████████████████████████████████| 2.8 MB 8.2 MB/s 
     |████████████████████████████████| 57 kB 5.0 MB/s 
     |████████████████████████████████| 806 kB 54.2 MB/s 
     |████████████████████████████████| 636 kB 39.2 MB/s 
     |████████████████████████████████| 408 kB 65.4 MB/s 
     |████████████████████████████████| 136 kB 42.2 MB/s 
     |████████████████████████████████| 829 kB 51.3 MB/s 
     |████████████████████████████████| 3.3 MB 43.5 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 895 kB 48.2 MB/s 
     |████████████████████████████████| 1.1 MB 4.3 MB/s 
     |████████████████████████████████| 96 kB 2.0 MB/s 
     |████████████████████████████████| 144 kB 48.7 MB/s 
     |████████████████████████████████| 271 kB 47.6 MB/s 
     |████████████████████████████████| 94 kB 327 kB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha2

In [2]:
#체크포인트 로드
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval()

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [3]:
#토크나이저 로드
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
#모델 입력값 만들기
input_ids = tokenizer.encode("안녕하세요", return_tensors="pt")

In [5]:
input_ids

tensor([[25906,  8702,  7801,  8084]])

In [6]:
#그리드 서치
import torch
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=False,
      min_length=10,
      max_length=50,
  )

In [7]:
#토큰 인덱스를 문장으로 복원하기
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



In [8]:
#빔 서치
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        mix_length=10,
        max_length=50,
        num_beams=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


In [9]:
#반복 줄이기(1)
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        no_repeat_ngram_size=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?" 하고 나는 물었다.
"그건 뭐죠?" 나는 물었다.
나는 대답하지 않았다.
"그런데 왜 그걸 물어요? 그건 무슨 뜻이에요?


## - - 이전 코드 끝 - -

또한 ***리피티션 페널티(repetition penalty)*** 라는 방식으로 반복을 통제할 수도 있습니다. `repetition_penalty`라는 인자를 주면 됩니다. 그 값이 1.0보다 클수록 페널티가 세게 적용됩니다. 다음 코드처럼 `repetition_penalty`값을 1.0으로 주면 아무런 페널티도 적용되지 않아서 그리디 서치와 똑같은 효과를 냅니다.  

cf) 리피티션 페널티는 모델의 출력 로짓(logit, 소프트맥스 변환 전 벡터)을 일부 가공해 적용합니다. 로짓 요솟값이 양수이면 원래 요솟값을 repetition_penalty로 나누고, 음수일 때는 repetition_penalty을 곱합니다. 자세한 내용은 해당 논문을 참고하시기를 바랍니다.

In [10]:
#반복 줄이기(2)
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



앞 코드에서 `repetition_penalty`값을 점점 높여 페널티를 세게 적용해보면 다음과 같습니다. 반복이 점점 줄어드는 경향이 있지만 여러 번 수행해도 생성 결과가 바뀌진 않습니다(`do_sample=False`).

In [11]:
#repetition_penalty=1.1
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"아니요, 저는요."
"그럼, 그건 무슨 말씀이신지요?"
"그럼, 그건 뭐예요?"



In [12]:
#repetition_penalty=1.2
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.2,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요."
"그래서 오늘은 제가 할 수 있는 일이 무엇인지 말해 보겠습니다."
"이제


In [13]:
#repetition_penalty=1.5
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


<**5단계**> **탑-k 샘플링하기**

지금까지 살펴본 문장 생성 방식은 모델이 출력한 다음 토큰 확률 분포를 점수로 활용한 것입니다. 전체 어휘 가운데 점수가 가장 높은 토큰을 다음 토큰으로 결정하는 방식입니다. 이렇게 하면 같은 모델에 같은 컨텍스트를 입력하는 경우 문장 생성을 여러 번 반복해도 그 결과는 같습니다.

그런데 ***샘플링(sampling)*** 이라는 방식도 있습니다. 샘플링 방식 예를 든 다음 그림을 보면 `그`라는 컨텍스트를 입력했을 때 모델은 다음 토큰으로 `집`(0.5), `책`(0.4), `사람`(0.1)이 그럴듯하다고 예측했습니다. 여기에서 다음 토큰을 확률적으로 선택합니다. `집`이 선택될 가능성이 50%로 제일 크고 `사람`이 선택될 가능성도 10%로 작지만, 없지는 않습니다.

그림에서 실제 선택된 다음 토큰은 `사람`입니다.   
이처럼 샘플링 방식으로 다음 토큰을 선택하게 된다면 같은 모델, 같은 컨텍스트라 하더라도 수행 때마다 문장 생성 결과가 다를 수 있습니다.

<center><그림 - 샘플링 방식></center>

<p align="center"><img src="https://i.imgur.com/it5cvUT.jpg">


***탑-k 샘플링(top-k sampling)***은 모델이 예측한 다음 토큰 확률 분포에서 확률값이 가장 높은 `k`로 토큰 가운데 하나를 다음 토큰으로 선택하는 기법입니다.   
탑-k 샘플링 방식 예를 든 다음 그림은 컨텍스트를 `그, k`를 6으로 뒀을 때 샘플링 대상 토큰들을 나타낸 것입니다. `책`처럼 확률값이 큰 단어가 다음 토큰으로 뽑힐 가능성이 높지만, `k`개 안에 있는 토큰이라면 `의자`같이 확률값이 낮은 케이스도 다음 토큰으로 추출될 수 있습니다.   
따라서 탑-k 샘플링은 매 수행 때마다 생성 결과가 달라집니다.

<center><그림 - 탑-k 샘플링(1)></center>

<p align="center"><img src="https://i.imgur.com/9UMEQsl.jpg">

위 그림에서 선택된 다음 단어가 `사람`이라고 가정해 봅시다.   
다음 그림은 기존 컨텍스트(`그`)에 이번에 선택된 `사람`을 이어붙인 새로운 컨텍스트인 `그 사람`을 모델에 입력해 다음 토큰 확률 분포를 계산하고 이를 내림차순으로 정렬한 것입니다. 우리는 k를 6으로 뒀으므로 `처럼`부터 `과`까지의 6개가 다음 토큰 후보가 됩니다.

<center><그림 - 탑-k 샘플링(2)></center>

<p align="center"><img src="https://i.imgur.com/nxeDQ1n.jpg">

다음 코드는 탑-k 샘플링을 수행하는 코드입니다. 핵심 인자는 `do_sample=True`, `top_k=50`입니다. 샘플링 방식으로 다음 단어를 선택하되 `k`를 50으로 설정한다는 뜻입니다. `top_k`는 1보다 큰 정수를 입력해야 합니다. 

만약 `top_k`를 1로 입력한다면 `do_sample` 인자를 `True`로 두더라도 그리디 서치와 똑같은 효과를 냅니다. 확률값이 가장 높은 1개 후보만 남긴 후 여기에서 하나를 뽑는 것이므로 매 순간 최선을 선택하는 것과 동일합니다.

In [14]:
#탑-k 샘플링 1try
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=True,
      min_length=10,
      max_length=50,
      top_k=50,
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요’ 같은 글들도 많이 올라온 상태"라며 “어떤 일이 있어도 우리를 지켜봐주신 분들을 생각하면 미안하다”고 사과했다.
이날 오후 2시 15분께 서울 종로구 사직동 대한문 뒤편에서 벌어진


In [15]:
#탑-k 샘플링 2try
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=True,
      min_length=10,
      max_length=50,
      top_k=50,
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요!!
- http://downline.
- http://www.eurechi.com/kr-commerce에서 검색 가능한
#선동후미챌


위 코드는 실행할 때마다 다른 문장이 생성됩니다. 두 번 수행한 결과 다른 문장이 생성된 것을 확인할 수 있었습니다.

그림 출처 - ratsgo